# Registration transform example

This example illustrates how to use a RegistrationTransform to temporally align the frames of an EOPatch using different algorithms

### Create cloudless timelapse

Imports from `sentinelhub` and `eolearn` to set up workflow that creates a timelapse

In [1]:
import datetime
import numpy as np

from sentinelhub import BBox, CRS, MimeType, DataCollection

from eolearn.core import EOPatch, FeatureType, LinearWorkflow
from eolearn.features import SimpleFilterTask
from eolearn.io import SentinelHubInputTask

Set up BBox of ROI and time interval

In [2]:
roi_bbox = BBox(bbox=[31.112895,29.957240,31.154222,29.987687], crs=CRS.WGS84)
# roi_bbox = BBox(bbox=[-6.57257, 37.2732, -5.728, 36.8549], crs=CRS.WGS84)
time_interval = ('2018-01-01', '2020-06-01')

This predicate function filters the images with a cloud coverage larger than a threshold to ensure images do not contain cloudy pixels

In [3]:
class MaxCCPredicate:
    def __init__(self, maxcc):
        self.maxcc = maxcc

    def __call__(self, img_cm):
        w, h, _ = img_cm.shape
        cc = np.sum(img_cm) / (w * h)
        return cc <= self.maxcc


Tasks of the workflow:
 * download S2 images (only 3 bands needed for true color visualitzation)
 * Download cloud mask (CLM) provided by Sentinel Hub  
 * filter out images with cloud coverage larger than a given threshold (e.g. 0.05)

In [4]:
download_task = SentinelHubInputTask(data_collection=DataCollection.SENTINEL2_L1C, 
                                     bands_feature=(FeatureType.DATA, 'BANDS'),
                                     resolution=10, 
                                     maxcc=0.5, 
                                     bands=['B02', 'B03', 'B04'], 
                                     time_difference=datetime.timedelta(hours=2),
                                     additional_data=[(FeatureType.MASK, 'dataMask', 'IS_DATA'), (FeatureType.MASK, 'CLM')]
                                    )
filter_clouds = SimpleFilterTask((FeatureType.MASK, 'CLM'), MaxCCPredicate(maxcc=0.05))

Build and execute timelapse as chain of transforms

In [5]:
timelapse = LinearWorkflow(download_task, filter_clouds)

result = timelapse.execute({
    download_task: {
        'bbox': roi_bbox,
        'time_interval': time_interval
    }
})

Get result as an eopatch

In [6]:
eopatch_clean = [result[key] for key in result.keys()][0]

In [7]:
eopatch_clean

EOPatch(
  data: {
    BANDS: numpy.ndarray(shape=(130, 331, 404, 3), dtype=float32)
  }
  mask: {
    CLM: numpy.ndarray(shape=(130, 331, 404, 1), dtype=uint8)
    IS_DATA: numpy.ndarray(shape=(130, 331, 404, 1), dtype=bool)
  }
  scalar: {}
  label: {}
  vector: {}
  data_timeless: {}
  mask_timeless: {}
  scalar_timeless: {}
  label_timeless: {}
  vector_timeless: {}
  meta_info: {
    maxcc: 0.5
    service_type: 'processing'
    size_x: 404
    size_y: 331
    time_difference: datetime.timedelta(seconds=7200)
    time_interval: (datetime.datetime(2018, 1, 1, 0, 0), datetime.datetime(2020, 6, 1, 23, 59, 59))
  }
  bbox: BBox(((31.112895, 29.95724), (31.154222, 29.987687)), crs=CRS('4326'))
  timestamp: [datetime.datetime(2018, 1, 4, 8, 33, 28), ..., datetime.datetime(2020, 5, 28, 8, 41, 58)], length=130
)

#### Help function to create GIFs

In [8]:
import imageio, os

def make_gif(eopatch, project_dir, filename, fps):
    """
    Generates a GIF animation from an EOPatch.
    """
    with imageio.get_writer(os.path.join(project_dir, filename), mode='I', fps=fps) as writer:
            for image in eopatch:
                writer.append_data(np.array(np.clip(2.8*image[..., [2, 1, 0]], 0, 255), dtype=np.uint8))

Write clean EOPatch to GIF

In [9]:
make_gif(eopatch=eopatch_clean.data['BANDS'] * 255, project_dir='.', filename='eopatch_clean.gif', fps=3)

## Run registrations

Import registrations

In [11]:
from eolearn.coregistration import ECCRegistration, ThunderRegistration, PointBasedRegistration

The constructor of the Registration objects takes the attribute type, field name and index of the channel to be used for registration, a dictionary specifying the parameters of the registration, and the interpolation method to be applied to the images. The interpolation methods are (NEAREST, LINEAR and CUBIC). Default is CUBIC. A nearest neighbour interpolation is used on ground-truth data to avoid creation of new labels.

### Thunder registration

This algorithm computes translations only between pairs of images, using correlation on the Fourier transforms of the images

In [13]:
coregister_thunder = ThunderRegistration((FeatureType.DATA, 'BANDS'), channel=2)

eopatch_thunder = coregister_thunder(eopatch_clean)

Write result to GIF

In [14]:
make_gif(eopatch=eopatch_thunder.data['BANDS']*255, project_dir='.', filename='eopatch_thunder.gif', fps=3)

### Enhanced Cross-Correlation in OpenCV

This algorithm uses intensity values to maximise cross-correlation between pair of images. It uses an Euler transformation (x,y translation plus rotation). 

In [15]:
params = {'MaxIters': 200}
coregister_ecc = ECCRegistration((FeatureType.DATA, 'BANDS'), channel=2, params=params)

eopatch_ecc = coregister_ecc(eopatch_clean)

In [16]:
make_gif(eopatch=eopatch_ecc.data['BANDS']*255, project_dir='.', filename='eopatch_ecc.gif', fps=3)

### Point-Based Registration in OpenCV

Three transformation models are supported for point-based registration, i.e. Euler, PartialAffine and Homography. These methods compute feature descriptors (i.e. SIFT or SURF) of the pair of images to be registered, and estimate a robust transformation using RANSAC to align the matching points. These methods perform poorly compared to the other methods due to the inaccuracies of the feature extraction, point-matching and model fitting. If unplausible transformations are estimated, a warning is issued and an identity matrix is employed instead of the estimated transform. Default parameters are (Model=Euler, Descriptor=SIFT, RANSACThreshold=7.0, MaxIters=1000).

Note: In case the following cell will raise an error

```Python
AttributeError: module 'cv2.cv2' has no attribute 'xfeatures2d'
```

uninstall and reinstall Python package `opencv-contrib-python`

In [19]:
params = {
    'Model': 'Euler',
    'Descriptor': 'SURF',
    'RANSACThreshold': 7.0,
    'MaxIters': 1000
}

coregister_pbased = PointBasedRegistration((FeatureType.DATA, 'BANDS'), channel=2, params=params)

eopatch_pbased = coregister_pbased(eopatch_clean)

In [20]:
make_gif(eopatch=eopatch_pbased.data['BANDS']*255, project_dir='.', filename='eopatch_pbased.gif', fps=3)